In [1]:

#cost function
def evaluate(assign, size):
    total = 0
    for i in range(size):
        for j in range(size):
            #if i!=j:
            total += flows[i][j]*distances[assign[i]][assign[j]]
    return total

#take first better solution and move in that direction
def local_search_first(val, curr, size):
    for i in range(size):
        for j in range(size):
            if i<j:
                temp = curr.copy()
                temp[i] = curr[j]
                temp[j] = curr[i]
                new_val = evaluate(temp,size)
                if new_val < val:
                    return new_val, temp
    return val, curr

# import csv
# fname = "qapdata/bur26a.dat"
# # read flash.dat to a list of lists
# data = [list(map(int, i.split())) for i in open(fname,'r').readlines()]
# for row in data:
#     for item in row:
#         item = int(item)
# size = data[0][0]
# flows = data[2:2+size]
# distances = data[3+size:len(data)-1]
# # print(len(flows),len(distances))
# # print(flows)
# # print(distances)
# current = [i for i in range(size)]
# val = evaluate(current, size)
# print(val,current)
# for i in range(100):
#     val, current = local_search_first(val, current, size)
# print(val,current)
# print(evaluate(current,size))

In [2]:
import random

# get random neighbour
def get_neighbour(perm, value, n, depth):
    for i in range(depth):
        swap1 = random.randrange(0, n)
        swap2 = random.randrange(0, n)
        while swap2 == swap1:
            swap2 = random.randrange(0, n)
        neighbour = perm.copy()
        neighbour[swap1] = perm[swap2]
        neighbour[swap2] = perm[swap1]
        perm = neighbour.copy()
    #             Compute new cost
    cost = evaluate(neighbour, n)
    #TO DO efficient cost
    #cost = value-
    return neighbour, cost
    

In [3]:
import numpy as np
#basic tabu with only short term
def tabu_search( perm,size, Tnum):
    MAX_TABU_SIZE = 10
    WALK_LEN = 1
    best = perm.copy()
    bestCost = evaluate(best, size)
    tabuList = [best]
    for i in range(Tnum):
        neighbour, cost = local_search_best_tabu(perm, size, tabuList)
#         print("perm:", perm, "nwighbour:",  neighbour)
#         print("\n\n")
#         print(tabuList)
#         print("\n\n")
        perm = neighbour.copy()
        tabuList = np.append(tabuList, [neighbour], axis=0)
        if len(tabuList)>MAX_TABU_SIZE:
            if np.all(tabuList[0]==neighbour):#will this ever hit?
                break
            tabuList = np.delete(tabuList, 0, axis=0)
        if cost<bestCost:
            best = neighbour.copy()
            bestCost = cost
    return best
current = sorted([i for i in range(size)], reverse = True)
result=tabu_search( current,len(current), 10)
print(evaluate(result, size), result)
result=tabu_search( current,len(current), 100)
print(evaluate(result, size), result)
result=tabu_search( current,len(current), 1000)
print(evaluate(result, size), result)

NameError: name 'size' is not defined

In [12]:
#best in neighbourhood of swaps=1 where best is not in tabu
def local_search_best_tabu(curr, size, tabu):
    cBest = np.append([curr[1],curr[0]],curr[2:])
    n = size
    cVal = evaluate(cBest,size)
    while np.any(np.all(cBest == tabu)):
        swap1 = random.randrange(0, n)
        swap2 = random.randrange(0, n)
        while swap2 == swap1:
            swap2 = random.randrange(0, n)
        neighbour = curr.copy()
        neighbour[swap1] = curr[swap2]
        neighbour[swap2] = curr[swap1]
        cBest = neighbour.copy()
        cVal = evaluate(cBest,size)
#     print("start", cBest,"start")
    for i in range(size):
        for j in range(size):
            if i<j:
                temp = curr.copy()
                temp[i] = curr[j]
                temp[j] = curr[i]
                #implement speed up
                new_val = evaluate(temp,size)
#                 print(temp, tabu)
                if new_val < cVal and not np.any(np.all(temp == tabu)):
                    cBest = temp.copy()
                    cVal = new_val
    return cBest, cVal

p2 = np.array([0, 1, 2, 5, 4, 3, 6, 7, 8,9, 10, 11, 12, 13, 14, 21, 16, 17, 18, 19, 20, 15, 22, 23, 24, 25])
p1 = np.array([1, 0, 2, 5, 4, 3, 6, 7, 8,9, 10, 11, 12, 13, 14, 21, 16, 17, 18, 19, 20, 15, 22, 23, 24, 25])
local_search_best_tabu(p1, 26, [p2])

(array([ 1,  0,  2,  5,  4,  3,  6,  7,  8,  9, 10, 14, 12, 13, 11, 21, 16,
        17, 18, 19, 20, 15, 22, 23, 24, 25]),
 1452056.0)

In [ ]:
def evaluate_swap(assign,s1, s2, size):
    total = 0
    for i in range(size):
        for j in range(size):
            #if i!=j:
            total += flows[i][j]*distances[assign[i]][assign[j]]
p1 = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8,9, 10, 11, 12, 13, 14, 21, 16, 17, 18, 19, 20, 15, 22, 23, 24, 25])
#swapping 3 and 5
p2 = np.array([0, 1, 2, 5, 4, 3, 6, 7, 8,9, 10, 11, 12, 13, 14, 21, 16, 17, 18, 19, 20, 15, 22, 23, 24, 25])
-flows[3][5]*distances[3][5]
-flows[5][3]*distances[5][3]
+flows[5][3]*distances[3][5]
+flows[3][5]*distances[5][3]
-flows[3][4]*distances[3][4]
+flows[3][4]*distances[5][4]

In [4]:
np.any(np.all(p2 == [p1]))

NameError: name 'p2' is not defined

In [5]:
curr = np.array(current)
tabu = [p2]
cBest = np.array(np.append([curr[1],curr[0]],curr[2:]))
print(cBest,tabu)
n = size
cVal = evaluate(cBest,size)
while np.any(np.all(cBest == tabu, axis=1)):
    swap1 = random.randrange(0, n)
    swap2 = random.randrange(0, n)
    while swap2 == swap1:
        swap2 = random.randrange(0, n)
    neighbour = curr.copy()
    neighbour[swap1] = perm[swap2]
    neighbour[swap2] = perm[swap1]
    cBest = neighbour.copy()
    cVal = evaluate(cBest,size)

NameError: name 'current' is not defined

In [6]:
def read_instance(fname):
    data = np.array([])
    for i in open(fname,'r').readlines():
        data = np.append(data, i.split())
    size = int(data[0])
    flows = np.zeros(shape=(size, size))
    for i in range(size):
        row = np.array([])
        for j in range(size):
            flows[i][j] = data[1+i*size+j]
    distances = np.zeros(shape=(size, size))
    base = 1+size**2
    for i in range(size):
        row = np.array([])
        step = i*size
        for j in range(size):
            distances[i][j] = data[base+step+j]
    return size, flows, distances


In [7]:
# 5482686 [21, 1, 2, 11, 12, 23, 6, 8, 7, 20, 19, 3, 4, 13, 10, 14, 0, 17, 18, 15, 9, 16, 22, 5, 24, 25]
# 5435743 [22, 10, 25, 12, 1, 24, 14, 0, 3, 7, 19, 6, 11, 17, 13, 20, 8, 4, 18, 15, 5, 9, 21, 2, 23, 16]
# 5435743 [22, 10, 25, 12, 1, 24, 14, 0, 3, 7, 19, 6, 11, 17, 13, 20, 8, 4, 18, 15, 5, 9, 21, 2, 23, 16]


In [16]:
# analysis
import os
import random
import numpy as np
directory = os.fsencode("qapdata")
counter = 0
for file in os.listdir(directory):
#     counter += 1
#     if counter>10:
#         break
    filename = os.fsdecode(file)
    #if filename.endswith(".dat") and (filename.startswith("tai") or filename.startswith("bur")):
    if filename.endswith(".dat") and filename.startswith("bur"):
    #if filename.endswith(".dat"): 
        fname = "qapdata/"+filename
        # read flash.dat to a list of lists
        
        size , flows, distances = read_instance(fname)
        current = np.random.permutation(size).astype(int)
        #print(current, size, flows, distances)
        val = evaluate(current, size)
#         print("\n initial solution for "+filename+": ")
#         print(val,current)
        result = tabu_search( current,size, 10)
#         for count in range(10):
#             val, current = local_search_first(val, current, size)
#         result = current
        val = evaluate(result, size)
#         print("\n tabu solution for "+filename+": ")
#         print(val,current)
        fname = "qapsoln/"+filename[:-3]+"sln"
        try:
            data = [list(map(int, i.split())) for i in open(fname,'r').readlines()]
            print("\n gap for "+filename+": ")
            try:
                print(100*(val/data[0][1]-1),"\n")
            except:
                print("\n optimum is 0 \n")
        except:
            print("no soln found\n")
        
        




 gap for bur26a.dat: 
0.739403722724985 


 gap for bur26b.dat: 
0.8339506088764104 


 gap for bur26c.dat: 
0.8309324380228134 


 gap for bur26d.dat: 
1.3638296619539636 


 gap for bur26e.dat: 
0.651137699584492 


 gap for bur26f.dat: 
1.5822132159224012 


 gap for bur26g.dat: 
0.8414110188103896 


 gap for bur26h.dat: 
0.6997378941202781 

